In [2]:
import pandas as pd
import numpy as np

In [20]:
df = pd.DataFrame({'animal': 'cat dog cat fish dog cat cat'.split(),
                  'size':list('SSMMMLL'),
                  'weight': [8, 10, 11, 1, 20, 12, 12],
                  'adult' : [False] * 5 + [True] * 2})
print(df)
df.groupby('animal').apply(lambda subf: subf['size'][subf['weight'].idxmax()])

   adult animal size  weight
0  False    cat    S       8
1  False    dog    S      10
2  False    cat    M      11
3  False   fish    M       1
4  False    dog    M      20
5   True    cat    L      12
6   True    cat    L      12


animal
cat       L
dog       M
fish      M
dtype: object

In [23]:
gb = df.groupby('animal')
gb.get_group('cat')

,adult,animal,size,weight
0,False,cat,S,8
2,False,cat,M,11
5,True,cat,L,12
6,True,cat,L,12


In [25]:
def GrowUp(x):
    avg_weight =  sum(x[x['size'] == 'S'].weight * 1.5)
    avg_weight += sum(x[x['size'] == 'M'].weight * 1.25)
    avg_weight += sum(x[x['size'] == 'L'].weight)
    avg_weight /= len(x)
    return pd.Series(['L',avg_weight,True], index=['size', 'weight', 'adult'])
expected_df = gb.apply(GrowUp)
expected_df

,size,weight,adult
animal,,,
cat,L,12.4375,True
dog,L,20.0000,True
fish,L,1.2500,True


In [44]:
import functools
S = pd.Series([i/100.0 for i in range(1,11)])
print(S)
def CumRet(x, y):
    print("%f %f" % (x,y))
    return x*(1+y)

def Red(x):
    print("in Red")
    print(x)
    return functools.reduce(CumRet,x,1.0)

pd.expanding_apply(S, Red)

0    0.01
1    0.02
2    0.03
3    0.04
4    0.05
5    0.06
6    0.07
7    0.08
8    0.09
9    0.10
dtype: float64
in Red
[ 0.01]
1.000000 0.010000
in Red
[ 0.01  0.02]
1.000000 0.010000
1.010000 0.020000
in Red
[ 0.01  0.02  0.03]
1.000000 0.010000
1.010000 0.020000
1.030200 0.030000
in Red
[ 0.01  0.02  0.03  0.04]
1.000000 0.010000
1.010000 0.020000
1.030200 0.030000
1.061106 0.040000
in Red
[ 0.01  0.02  0.03  0.04  0.05]
1.000000 0.010000
1.010000 0.020000
1.030200 0.030000
1.061106 0.040000
1.103550 0.050000
in Red
[ 0.01  0.02  0.03  0.04  0.05  0.06]
1.000000 0.010000
1.010000 0.020000
1.030200 0.030000
1.061106 0.040000
1.103550 0.050000
1.158728 0.060000
in Red
[ 0.01  0.02  0.03  0.04  0.05  0.06  0.07]
1.000000 0.010000
1.010000 0.020000
1.030200 0.030000
1.061106 0.040000
1.103550 0.050000
1.158728 0.060000
1.228251 0.070000
in Red
[ 0.01  0.02  0.03  0.04  0.05  0.06  0.07  0.08]
1.000000 0.010000
1.010000 0.020000
1.030200 0.030000
1.061106 0.040000
1.103550 0.050000
1.1

0    1.010000
1    1.030200
2    1.061106
3    1.103550
4    1.158728
5    1.228251
6    1.314229
7    1.419367
8    1.547110
9    1.701821
dtype: float64

In [49]:
df = pd.DataFrame({'A':[1,1,2,2], 'B':[1,-1,1,2]})
gb = df.groupby('A')
print(type(gb))
def replace(g):
    print(g)
    mask = g<0
    g.loc[mask] = g[~mask].mean()
    return g
gb.transform(replace)

<class 'pandas.core.groupby.DataFrameGroupBy'>
0    1
1   -1
Name: B, dtype: int64
   B
0  1
1  1
2    1
3    2
Name: B, dtype: int64


,B
0,1
1,1
2,1
3,2


In [52]:
df = pd.DataFrame({'code':['foo', 'bar', 'baz']*2,
                 'data': [0.16, -0.21, 0.33, 0.45, -0.59, 0.62],
                 'flag':[False, True]*3})
code_groups = df.groupby('code')
print(code_groups[['data']].transform(sum))
agg_n_sort_order = code_groups[['data']].transform(sum).sort('data')
print(agg_n_sort_order)
sorted_df = df.ix[agg_n_sort_order.index]
sorted_df

   data
0  0.61
1 -0.80
2  0.95
3  0.61
4 -0.80
5  0.95
   data
1 -0.80
4 -0.80
0  0.61
3  0.61
2  0.95
5  0.95


,code,data,flag
1,bar,-0.21,True
4,bar,-0.59,False
0,foo,0.16,False
3,foo,0.45,True
2,baz,0.33,False
5,baz,0.62,True
